## Vase Problem

In [22]:
from abc import ABCMeta, abstractmethod
import queue as Q
import numpy as np
# Basic data structure Stack
#
#
class Stack:
     def __init__(self):
         self.items = []
     def isEmpty(self):
         return self.items == []
     def push(self, item):
         self.items.append(item)
     def pop(self):
         return self.items.pop()
     def peek(self):
         return self.items[len(self.items)-1]
     def size(self):
         return len(self.items)
#
# Basic data structure Queue
# (Use my own Queue instead of import) 
#        
class Queue:
    def __init__(self):
        self.items = []
    def isEmpty(self):
        return self.items == []
    def add(self, item):
        self.items.insert(0,item)
    def get(self):
        return self.items.pop()
    def size(self):
        return len(self.items)
    
class Solver(metaclass=ABCMeta):
    """
     ABC=Abstract base class
     Abstract Solver class that searches through the state space, called by the Board class
    """
    @abstractmethod
    def get(self):
         raise NotImplementedError()
    @abstractmethod
    def add(self):
         raise NotImplementedError()
     
class Solver1(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     uses a Queue
    """
    def __init__(self):
        self.queue=Queue()
    def get(self):
        return self.queue.get()
    def add(self,board):
        self.queue.add(board)

class Solver2(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     Uses a Stack
    """
    def __init__(self):
        self.stack=Stack()
    def get(self):
        return self.stack.pop()
    def add(self,board):
        self.stack.push(board)

class Solver3:
    def __init__(self):
        self.pq = Q.PriorityQueue()
        self.goal = np.array([0, 8, 8, 8])

    def get(self):
        if not self.pq.empty():
            (val, board) = self.pq.get()
            return np.array(board), val
        return None, None

    def add(self, board):
        value = self.heuristic(board)
        self.pq.put((value, board.tolist()))

    def heuristic(self, board):
       return abs(8 - board[1]) + abs(8 - board[2]) + abs(8 - board[3])

### Build the Board


In [18]:
class Board:
    def __init__(self):
        self.total_balsam = 24
        self.vessels = [5, 11, 13]
        self.board = np.array([24, 0, 0, 0])  # Initial state: [Remaining, Vessel1, Vessel2, Vessel3]
        self.goal = np.array([0, 8, 8, 8])  # Goal state
        self.used = []
        self.solver = Solver3()

    def solve(self):
        self.used = []
        self.solver.add(self.board)
        board, heuristic_value = self.solver.get()
        step = 0
        max_steps = 1000

        print("Vase Problem Solution Process:")
        print("==============================")

        while board is not None and step < max_steps:
            step += 1
            
            print(f"\nStep {step}:")
            print(f"Exploring State: {board}")
            self.printBoard(board)
            print(f"Heuristic Value: {heuristic_value}")
            
            if np.array_equal(board, self.goal):
                print("\n==============================")
                print("Solution found!")
                print(f"Total steps: {step}")
                return
            
            moves = self.getMoves(board)
            print("Valid Moves:")
            for i, move in enumerate(moves):
                from_vessel = "Remaining" if move[0] == 0 else f"Vessel {move[0]}"
                to_vessel = "Remaining" if move[1] == 0 else f"Vessel {move[1]}"
                print(f"  {i+1}. From {from_vessel} to {to_vessel}: {move[2]} oz")
            
            new_states_added = 0
            for move in moves:
                new_board = self.makeMove(board, move)
                if self.isNotUsed(new_board):
                    self.solver.add(new_board)
                    self.used.append(new_board)
                    new_states_added += 1
            
            print(f"New states added: {new_states_added}")
            
            board, heuristic_value = self.solver.get()
        
        print("\n==============================")
        if step >= max_steps:
            print(f"Search terminated after {max_steps} steps without finding a solution.")
        else:
            print("No solution found.")

    def getMoves(self, board):
        moves = []
        remaining, v1, v2, v3 = board
        
        # Pour from remaining to each vessel
        for i, capacity in enumerate(self.vessels):
            amount = min(remaining, capacity - board[i+1])
            if amount > 0:
                moves.append((0, i+1, amount))
        
        # Pour from each vessel to another or back to remaining
        for i in range(1, 4):
            for j in range(4):
                if i != j:
                    amount = min(board[i], self.vessels[j-1] - board[j]) if j != 0 else board[i]
                    if amount > 0:
                        moves.append((i, j, amount))
        
        return moves

    def makeMove(self, board, move):
        new_board = board.copy()
        from_vessel, to_vessel, amount = move
        new_board[from_vessel] -= amount
        new_board[to_vessel] += amount
        return new_board

    def isValidMove(self, board, move):
        from_vessel, to_vessel, amount = move
        if from_vessel == 0:
            return board[0] >= amount and self.vessels[to_vessel-1] - board[to_vessel] >= amount
        else:
            return board[from_vessel] >= amount and (to_vessel == 0 or self.vessels[to_vessel-1] - board[to_vessel] >= amount)

    def isNotUsed(self, board):
        return all(not np.array_equal(board, u) for u in self.used)

    def printBoard(self, board):
        print(f"Remaining: {board[0]} | Vessel 1 (5oz): {board[1]} | Vessel 2 (11oz): {board[2]} | Vessel 3 (13oz): {board[3]}")

### Run and Test

In [21]:
def main():
    game = Board()
    game.solve()

if __name__ == '__main__':
    main()

Vase Problem Solution Process:

Step 1:
Exploring State: [24  0  0  0]
Remaining: 24 | Vessel 1 (5oz): 0 | Vessel 2 (11oz): 0 | Vessel 3 (13oz): 0
Heuristic Value: 32
Valid Moves:
  1. From Remaining to Vessel 1: 5 oz
  2. From Remaining to Vessel 2: 11 oz
  3. From Remaining to Vessel 3: 13 oz
New states added: 3

Step 2:
Exploring State: [13  0 11  0]
Remaining: 13 | Vessel 1 (5oz): 0 | Vessel 2 (11oz): 11 | Vessel 3 (13oz): 0
Heuristic Value: 23
Valid Moves:
  1. From Remaining to Vessel 1: 5 oz
  2. From Remaining to Vessel 3: 13 oz
  3. From Vessel 2 to Remaining: 11 oz
  4. From Vessel 2 to Vessel 1: 5 oz
  5. From Vessel 2 to Vessel 3: 11 oz
New states added: 5

Step 3:
Exploring State: [ 0  0 11 13]
Remaining: 0 | Vessel 1 (5oz): 0 | Vessel 2 (11oz): 11 | Vessel 3 (13oz): 13
Heuristic Value: 16
Valid Moves:
  1. From Vessel 2 to Remaining: 11 oz
  2. From Vessel 2 to Vessel 1: 5 oz
  3. From Vessel 3 to Remaining: 13 oz
  4. From Vessel 3 to Vessel 1: 5 oz
New states added: 2

